# Generating LaTeX code for result tables

A very simple notebook that we use to combined our X-ray measurement results into a table formatted for the paper.

## Import Statements

In [1]:
import pandas as pd
import numpy as np

## Read the sample file

In [2]:
samp = pd.read_csv('../../sample_files/init_group_info.csv')
samp = samp.rename(columns={'est_ra': 'ra', 'est_dec': 'dec'})
samp

,name,ra,dec,redshift
0,25124A,243.629055,26.730240,0.186
1,25124B,243.504000,26.691000,0.186
2,25124C,243.541000,26.648000,0.186
3,44739,229.793620,28.331750,0.118
4,19670AB,150.226402,50.800565,0.134
5,19670A,150.256710,50.793942,0.134
6,19670B,150.214920,50.805014,0.134
7,19670C,150.065000,50.746000,0.134
8,19670D,150.168000,50.686000,0.134
9,12833A,129.496800,44.248700,0.145


## Read the result files

In [3]:
results = pd.read_csv('../../outputs/results/ltr_r500_pipeline_results.csv')
results_r2500 = pd.read_csv('../../outputs/results/ltr_r2500_pipeline_results.csv')

results_froztemp = pd.read_csv('../../outputs/results/ltr_r500_froztemp_pipeline_results.csv')
results_r2500_froztemp = pd.read_csv('../../outputs/results/ltr_r2500_froztemp_pipeline_results.csv')

results_gm = pd.read_csv("../../outputs/results/all_hot_gm_measurements.csv")
results_gm = results_gm[['name', 'gm500', 'gm500-', 'gm500+', 'gm2500', 'gm2500-', 'gm2500+']]

results_gm_froztemp = pd.read_csv("../../outputs/results/all_hot_gm_measurements.csv")
results_gm_froztemp = results_gm_froztemp[['name', 'gm_froztemp500', 'gm_froztemp500-', 'gm_froztemp500+', 
                                           'gm_froztemp2500', 'gm_froztemp2500-', 'gm_froztemp2500+']]

## Combine the result dataframes

In [4]:
lat_r500_res = results.copy()

lat_r2500_res = results_r2500.copy()
del lat_r2500_res['redshift']
del lat_r2500_res['ra']
del lat_r2500_res['dec']

lat_r500_ft_res = results_froztemp.copy()

lat_r2500_ft_res = results_r2500_froztemp.copy()
del lat_r2500_ft_res['redshift']
del lat_r2500_ft_res['ra']
del lat_r2500_ft_res['dec']

comb = pd.merge(lat_r500_res, lat_r2500_res, on='name')
comb = pd.merge(comb, results_gm, how='outer')
comb_ft = pd.merge(lat_r500_ft_res, lat_r2500_ft_res, on='name')
comb_ft = pd.merge(comb_ft, results_gm_froztemp, how='outer')

In [5]:
comb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 41 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   name                 16 non-null     object 
 1   ra                   16 non-null     float64
 2   dec                  16 non-null     float64
 3   redshift             16 non-null     float64
 4   r500                 14 non-null     float64
 5   r500+-               14 non-null     float64
 6   Tx500                14 non-null     float64
 7   Tx500-               14 non-null     float64
 8   Tx500+               14 non-null     float64
 9   Lx500_0.5-2.0        14 non-null     float64
 10  Lx500_0.5-2.0-       14 non-null     float64
 11  Lx500_0.5-2.0+       14 non-null     float64
 12  Lx500_0.01-100.0     14 non-null     float64
 13  Lx500_0.01-100.0-    14 non-null     float64
 14  Lx500_0.01-100.0+    14 non-null     float64
 15  Tx500ce              13 non-null     float

In [6]:
comb_ft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 42 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   name                  16 non-null     object 
 1   ra                    16 non-null     float64
 2   dec                   16 non-null     float64
 3   redshift              16 non-null     float64
 4   r500                  14 non-null     float64
 5   r500+-                14 non-null     float64
 6   froz_Tx500            14 non-null     float64
 7   froz_Tx500+-          14 non-null     float64
 8   Lx500_0.5-2.0         14 non-null     float64
 9   Lx500_0.5-2.0-        14 non-null     float64
 10  Lx500_0.5-2.0+        14 non-null     float64
 11  Lx500_0.01-100.0      14 non-null     float64
 12  Lx500_0.01-100.0-     14 non-null     float64
 13  Lx500_0.01-100.0+     14 non-null     float64
 14  Lx500ce_0.5-2.0       14 non-null     float64
 15  Lx500ce_0.5-2.0-      14 

## Generating the LaTeX code

In [31]:
lines = ""
for row_ind, row in samp.iterrows():
    rel_name = row['name']
    rel_res = comb[comb['name'] == rel_name].iloc[0]
    rel_ft_res = comb_ft[comb_ft['name'] == rel_name].iloc[0]
    
    if rel_name == 'J083829.2+441904.8':
        rel_name = '\\shortnewrbsgrp{}'
#         print(rel_ft_res)
    elif rel_name == 'J215504.8-093520':
        rel_name = '\\shortphlgroup{}'
    elif rel_name == 'J215437.2-091532.4':
        rel_name = '\\shortnrthphlgroup{}'
    
    res_line = "{} &".format(rel_name)
    
    rad_temp = r" ${r}{{\pm}}{re}$ &"
    if not np.isnan(rel_res['r2500']):
        res_line += rad_temp.format(r=rel_res['r2500'].astype(int), re=rel_res['r2500+-'].astype(int))
    else:
        res_line += " - &"
        
    if not np.isnan(rel_res['r500']):
        res_line += rad_temp.format(r=rel_res['r500'].astype(int), re=rel_res['r500+-'].astype(int))
    else:
        res_line += " - &"
    
    # This isn't elegant or sophisticated, but I cannot be bothered to write something more general
    cols = [['Tx2500', 'Tx2500-', 'Tx2500+'], ['Tx500', 'Tx500-', 'Tx500+'], 
            ['Lx2500_0.5-2.0', 'Lx2500_0.5-2.0-', 'Lx2500_0.5-2.0+'], 
            ['Lx500_0.5-2.0', 'Lx500_0.5-2.0-', 'Lx500_0.5-2.0+'], 
            ['Lx500ce_0.5-2.0', 'Lx500ce_0.5-2.0-', 'Lx500ce_0.5-2.0+'], 
            ['gm2500', 'gm2500-', 'gm2500+'],
            ['gm500', 'gm500-', 'gm500+']]
    
    col_temp = r" ${v}_{{-{me}}}^{{+{pe}}}$ &"
    for col_set in cols:
        rel_val = rel_res[col_set].values
        
        if np.isnan(rel_val[0]):
            res_line += " - &"
            continue
        
        if col_set[0][0] == 'L':
            rel_val = rel_val / 1e+43
            
        res_line += col_temp.format(v=rel_val[0].round(2), 
                                    me=rel_val[1].round(2), 
                                    pe=rel_val[2].round(2))
        
    res_ft_line = " &"
    
    if not np.isnan(rel_ft_res['r2500']):
        res_ft_line += rad_temp.format(r=rel_ft_res['r2500'].astype(int), re=rel_ft_res['r2500+-'].astype(int))
    else:
        res_ft_line += " - &"
        
    if not np.isnan(rel_ft_res['r500']):
        res_ft_line += rad_temp.format(r=rel_ft_res['r500'].astype(int), re=rel_ft_res['r500+-'].astype(int))
    else:
        res_ft_line += " - &"
    
    # This isn't elegant or sophisticated, but I cannot be bothered to write something more general
    cols = [['froz_Tx2500', 'froz_Tx2500+-'], ['froz_Tx500', 'froz_Tx500+-'], 
            ['Lx2500_0.5-2.0', 'Lx2500_0.5-2.0-', 'Lx2500_0.5-2.0+'], 
            ['Lx500_0.5-2.0', 'Lx500_0.5-2.0-', 'Lx500_0.5-2.0+'], 
            ['Lx500ce_0.5-2.0', 'Lx500ce_0.5-2.0-', 'Lx500ce_0.5-2.0+'],
            ['gm_froztemp2500', 'gm_froztemp2500-', 'gm_froztemp2500+'],
            ['gm_froztemp500', 'gm_froztemp500-', 'gm_froztemp500+']]
    for col_set in cols:
        rel_val = rel_ft_res[col_set].values
        
        if np.isnan(rel_val[0]):
            res_ft_line += " - &"
            continue
        
        if col_set[0][0] == 'L':
            rel_val = rel_val / 1e+43
        
        if len(col_set) == 3:
            res_ft_line += col_temp.format(v=rel_val[0].round(2), 
                                           me=rel_val[1].round(2), 
                                           pe=rel_val[2].round(2))
        elif len(col_set) == 2:
            res_ft_line += rad_temp.format(r=rel_val[0].round(2), 
                                           re=rel_val[1].round(2))

    # Bodging the gas mass stuff until I move this to another notebook
#     res_line += ' - & -'
#     res_ft_line += ' - & -'
    
    # Adding the end of line characters
    res_line += '\\\ \n'
    res_ft_line += '\\\[2mm]\n%\n'
    
    lines += res_line
    lines += res_ft_line
    
print(lines) 

25124A & $365{\pm}12$ & $893{\pm}44$ & $3.48_{-0.21}^{+0.23}$ & $4.08_{-0.33}^{+0.42}$ & $5.63_{-0.07}^{+0.08}$ & $7.14_{-0.15}^{+0.14}$ & $4.38_{-0.13}^{+0.12}$ & $0.88_{-0.04}^{+0.05}$ & $2.8_{-0.16}^{+0.16}$ &\\ 
 & $405{\pm}6$ & $867{\pm}15$ & $4.19{\pm}0.09$ & $3.85{\pm}0.08$ & $5.89_{-0.09}^{+0.08}$ & $7.18_{-0.13}^{+0.12}$ & $4.48_{-0.11}^{+0.12}$ & $1.03_{-0.02}^{+0.02}$ & $2.7_{-0.06}^{+0.06}$ &\\m]
%
25124B & $281{\pm}19$ & $232{\pm}46$ & $2.14_{-0.27}^{+0.37}$ & $2.59_{-0.74}^{+2.4}$ & $0.34_{-0.02}^{+0.02}$ & $0.17_{-0.04}^{+0.04}$ & - & $0.09_{-0.01}^{+0.01}$ & $0.07_{-0.02}^{+0.02}$ &\\ 
 & $251{\pm}9$ & $531{\pm}22$ & $1.78{\pm}0.1$ & $1.7{\pm}0.1$ & $0.27_{-0.02}^{+0.02}$ & $0.49_{-0.06}^{+0.04}$ & $0.41_{-0.06}^{+0.05}$ & $0.08_{-0.01}^{+0.01}$ & $0.12_{-0.02}^{+0.03}$ &\\m]
%
25124C & $242{\pm}11$ & $494{\pm}20$ & $1.73_{-0.2}^{+0.32}$ & $1.41_{-0.08}^{+0.09}$ & $0.25_{-0.02}^{+0.03}$ & $0.68_{-0.05}^{+0.06}$ & $0.66_{-0.03}^{+0.06}$ & - & - &\\ 
 & - & - & - & - & - 